In [ ]:
%pip install xlrd mysql-connector-python pandas

import xlrd
import mysql.connector
import pandas as pd

# Conectarse a la base de datos MySQL
conexion = mysql.connector.connect(
    host="database-1.cjyy8w6ya3fr.eu-north-1.rds.amazonaws.com",

)

Note: you may need to restart the kernel to use updated packages.


In [2]:


nombre_base_datos = "sandbox"
nombre_tabla_ventas = "EHR_ventas"
nombre_tabla_calendario = "EHR_calendario"
nombre_tabla_pedidos = "EHR_pedidos"

cursor = conexion.cursor()
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_base_datos}")

cursor.execute(f"USE {nombre_base_datos}")

In [3]:
cursor.execute(f"DROP TABLE IF EXISTS {nombre_tabla_ventas}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {nombre_tabla_ventas} ("
               "FAMILIA VARCHAR(255),"
               "Tipo VARCHAR(255),"
               "FechaVenta DATE,"
               "HoraVenta INT,"
               "Articulo VARCHAR(255),"
               "Cantidad FLOAT,"
               "Precio FLOAT,"
               "Importe FLOAT"
               ")")

In [4]:
cursor.execute(f"DROP TABLE IF EXISTS {nombre_tabla_calendario}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {nombre_tabla_calendario} ("
               "Fecha DATE,"
               "Festivo VARCHAR(255)"
               ")")

In [5]:
cursor.execute(f"DROP TABLE IF EXISTS {nombre_tabla_pedidos}")
cursor.execute(f"CREATE TABLE IF NOT EXISTS {nombre_tabla_pedidos} ("
               "Tipo VARCHAR(255),"
               "Fecha DATE,"
               "Articulo VARCHAR(255),"
               "Cantidad FLOAT,"
               "Precio FLOAT,"
               "Importe FLOAT"
               ")")

In [6]:
%pip install mysql-connector-python xlrd pandas

import xlrd
import mysql.connector
import pandas as pd


Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install openpyxl

# Cargar el archivo Excel en Colab
nombre_archivo_ventas = "ArticulosPanaderia.xlsx"

# Abrir el archivo Excel
df = pd.read_excel(nombre_archivo_ventas)

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install openpyxl

import pandas as pd

# Cargar el archivo Excel en Colab
nombre_archivo_ventas = "ArticulosPanaderia.xlsx"

# Abrir el archivo Excel
df = pd.read_excel(nombre_archivo_ventas)

# Preparar los datos para la inserción en lotes
registros = []
for _, row in df.iterrows():
    familia = row['FAMILIA']
    tipo = row['Tipo']
    fecha_venta = row['FechaVenta'].strftime('%Y-%m-%d')
    hora_venta = row['HoraVenta']
    articulo = row['Articulo']
    cantidad = row['Cantidad']
    precio = row['Precio']
    importe = row['Importe']
    registros.append((familia, tipo, fecha_venta, hora_venta, articulo, cantidad, precio, importe))


Note: you may need to restart the kernel to use updated packages.


In [9]:
# Verificar que la variable está definida
print("nombre_tabla_ventas:", nombre_tabla_ventas)

# Verificar y reconectar si la conexión no está disponible
if not conexion.is_connected():
    conexion.reconnect(attempts=3, delay=2)
    cursor = conexion.cursor()

# Seleccionar la base de datos antes de insertar
cursor.execute(f"USE {nombre_base_datos}")

# Crear la consulta SQL para insertar los datos en la tabla
consulta = f"INSERT INTO {nombre_tabla_ventas} (FAMILIA, Tipo, FechaVenta, HoraVenta, Articulo, Cantidad, Precio, Importe) " \
           "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

# Ejecutar la consulta con la lista de registros en lotes
tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.execute(f"USE {nombre_base_datos}")
    cursor.executemany(consulta, lote_registros)
    conexion.commit()

nombre_tabla_ventas: EHR_ventas


In [10]:
# Cargar el archivo Excel en Colab
nombre_archivo_calendario = "Calendario.xlsx"

# Abrir el archivo Excel
df = pd.read_excel(nombre_archivo_calendario)

# Preparar los datos para la inserción en lotes
registros = []
for _, row in df.iterrows():
    fecha = row['Fecha'].strftime('%Y-%m-%d')
    festivo = row['Festivo']
    registros.append((fecha,festivo))

# Crear la consulta SQL para insertar los datos en la tabla
consulta = f"INSERT INTO {nombre_tabla_calendario} (Fecha, Festivo) " \
           "VALUES (%s, %s)"

# Ejecutar la consulta con la lista de registros en lotes
tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.executemany(consulta, lote_registros)
    conexion.commit()

In [11]:
# Cargar el archivo Excel en Colab
nombre_archivo_pedidos = "CantidadPedida.xlsx"

# Abrir el archivo Excel
df = pd.read_excel(nombre_archivo_pedidos)

# Preparar los datos para la inserción en lotes
registros = []
for _, row in df.iterrows():
    tipo = row['Tipo']
    fecha_venta = row['Fecha'].strftime('%Y-%m-%d')
    articulo = row['Articulo']
    cantidad = row['Cantidad']
    precio = row['Precio']
    importe = row['Importe']
    registros.append(( tipo, fecha_venta, articulo, cantidad, precio, importe))

# Crear la consulta SQL para insertar los datos en la tabla
consulta = f"INSERT INTO {nombre_tabla_pedidos} (Tipo, Fecha, Articulo, Cantidad, Precio, Importe) " \
           "VALUES (%s, %s, %s, %s, %s, %s)"

# Ejecutar la consulta con la lista de registros en lotes
tamano_lote = 1000
for i in range(0, len(registros), tamano_lote):
    lote_registros = registros[i:i + tamano_lote]
    cursor = conexion.cursor()
    cursor.executemany(consulta, lote_registros)
    conexion.commit()

In [12]:
# Nota: las consultas siguientes están preparadas para lanzarse con los datos cargados en la bbdd data y con los nombres iniciales, si se lanzan con usuario1 debae adaptarse

consulta_proceso="""


/*Ordenamos los productos por familia*/
drop table if exists sandbox.EHR_articulos_top;
create table sandbox.EHR_articulos_top as (

	select
	Articulo,
	FAMILIA,
	sum(importe) as importe_total,
	ROW_NUMBER () OVER(partition by FAMILIA order by sum(importe) desc) as orden
	from sandbox.EHR_ventas
	where FechaVenta >= '2021-05-01'
	group by 1,2

);


/*Creamos un calendario general del 1/1/2017 al 31/12/2023*/
drop table if exists sandbox.EHR_calendario_dias;
create table sandbox.EHR_calendario_dias as (

with recursive cte_calendario as (
	select date('2017-01-01') as calendar_date -- fecha de inicio
	union all
	select date_add(calendar_date, interval 1 day) as calendar_date from cte_calendario
	where date_add(calendar_date, interval 1 day) <= date('2023-12-31') -- condicion de fin, fecha final
)

select
calendar_date as fecha,
year(calendar_date) as fx_anno,
month(calendar_date) as fx_mes,
day(calendar_date) as fx_day,
date_format(calendar_date, '%Y%m') as fx_anno_mes,
date_format(calendar_Date,'%x-%v') as semana -- formato
from cte_calendario

);

/*Añadimos al calendario los festivos*/
drop table if exists sandbox.EHR_calendario_completo;
create table sandbox.EHR_calendario_completo as (
select
base.*,
festivos.festivo
from sandbox.EHR_calendario_dias base
left join
	(select
	a.*,
	row_number() over(partition by a.fecha order by a.festivo) as orden
	from sandbox.EHR_calendario a
	) festivos
on base.fecha=festivos.fecha
and festivos.orden=1
);

/*Agregamos las ventas por día (quitamos el detalle por hora)*/

/*Añadimos además el orden del producto y si está en el rango de fecha de estudio*/
drop table if exists sandbox.EHR_ventas_diarias;
create table sandbox.EHR_ventas_diarias as (
select
base.familia,
base.tipo,
base.fechaVenta,
calendario.festivo,
base.articulo,
sum(base.precio*base.cantidad)/sum(base.cantidad) as precio, /* Se calcula como el precio promedio del día, ya que hay cambios de precio a lo largo del día*/
articulos.orden as orden_articulo_familia,
case when base.fechaVenta >=date('2021-05-01') then 'S' else 'N' end as in_fecha_estudio,
sum(base.cantidad) as cantidad,
sum(base.importe) as importe

from sandbox.EHR_ventas base

inner join sandbox.EHR_calendario_completo calendario
on base.FechaVenta=calendario.fecha

inner join sandbox.EHR_articulos_top articulos
on base.familia=articulos.familia
and base.articulo=articulos.articulo


group by 1,2,3,4,5,7,8);


/*Creamos una vista final con el filtrado para el estudio*/
drop view if exists sandbox.EHR_ventas_diarias_estudio_completo;
create view sandbox.EHR_ventas_diarias_estudio_completo as
(select *
from sandbox.EHR_ventas_diarias
where tipo='VENTA'
and in_fecha_estudio='S'
and orden_articulo_familia<=5);


/*Creamos una vista extra con el filtrado para el estudio que no tenga días de mayo 23 (que habrá que predecir)*/
drop view if exists sandbox.EHR_ventas_diarias_estudio;
create view sandbox.EHR_ventas_diarias_estudio as
(select *
from sandbox.EHR_ventas_diarias
where tipo='VENTA'
and in_fecha_estudio='S'
and fechaventa < date('2023-05-01')
and orden_articulo_familia<=5);
"""

In [13]:
# Separar las consultas en una lista
consultas_lista = consulta_proceso.split(";")

# Eliminar espacios en blanco y elementos vacíos
consultas_lista = [consulta.strip() for consulta in consultas_lista if consulta.strip()]

# Ejecutar las consultas individualmente, omitiendo comentarios y vacíos
for consulta in consultas_lista:
    # Omitir comentarios de bloque y líneas vacías
    if consulta.startswith("/*") or consulta.startswith("--") or consulta == "":
        continue
    try:
        cursor.execute(consulta)
        conexion.commit()
    except Exception as e:
        print(f"Error ejecutando consulta:\n{consulta}\nError: {e}")
        break  # O puedes usar 'continue' si quieres seguir con las siguientes

Error ejecutando consulta:
create table sandbox.EHR_articulos_top as (

	select
	Articulo,
	FAMILIA,
	sum(importe) as importe_total,
	ROW_NUMBER () OVER(partition by FAMILIA order by sum(importe) desc) as orden
	from sandbox.EHR_ventas
	where FechaVenta >= '2021-05-01'
	group by 1,2

)
Error: 1050 (42S01): Table 'EHR_articulos_top' already exists


In [14]:
# cursor.execute(f"grant select on {nombre_base_datos}.ventas_diarias to usuario1")
# cursor.execute(f"grant select on {nombre_base_datos}.ventas_diarias_estudio to usuario1")

In [15]:
# Consultas adicionales por problema de permisos del usuario usuario1 en la base de datos data

In [16]:
import mysql.connector

# Reconnect if the connection is not alive
if not conexion.is_connected():
	conexion.reconnect(attempts=3, delay=2)
	cursor = conexion.cursor()

cursor.execute(f"drop table if exists sandbox.EHR_ventas_diarias_estudio;")

In [17]:
cursor.execute("DROP TABLE IF EXISTS sandbox.EHR_ventas_diarias_estudio")
cursor.execute("DROP TABLE IF EXISTS sandbox.EHR1_ventas_diarias_estudio")
cursor.execute(f"CREATE TABLE sandbox.EHR1_ventas_diarias_estudio AS SELECT * FROM {nombre_base_datos}.EHR_ventas_diarias_estudio")